In [61]:
import pandas as pd
from tqdm import tqdm

In [62]:
clv = True
NUM_THREAD = 3
STATUS = {0:"commit", 2:"abort"}
def get_thd(txn):
    return int(txn) % NUM_THREAD
def get_lock(lock_idx):
    lock_idx = int(lock_idx)
    if lock_idx == 0:
        return "SH"
    elif lock_idx == 1:
        return "EX"
    else:
        return "NONE"
def rm_txn(row, place, txn):
    thread = get_thd(txn)
    members = locktable[row][place]
    found = False
    for i, step in enumerate(members):
        if step[0] == txn:
            found = True
            break
    if not found:
        print("not found: {}".format(line))
        raise
    locktable[row][place] = members[:i] + members[i+1:]
    return thread, members[i]
def get_schedule(start, end, thread):
    return schedule.loc[start:end+1, thread][schedule.loc[start:end+1, thread].notnull()]
def find_locktable_with_txn(txn):
    for row in locktable:
        if txn in [item[0] for place in ['owners', 'waiters', 'retired'] for item in locktable[row][place] ]:
            print(row)
            print(locktable[row])
def find_locktable_with_row(row):
    print(locktable[row])

In [84]:
def parse(line):
    txn = ""
    tokens = line.split(" ")
    if "[thread]" in line:
        if "start running txn" in line:
            thread = int(tokens[2])
            txn = tokens[6]
            schedule.loc[ts, thread] = "start txn %s"%txn
            if clv:
                barriers[txn] = 0
        elif "finish txn" in line:
            txn = tokens[3]
            thread = get_thd(txn)
            rc = tokens[-1]
            # "[thread-%lu] finish txn %lu (aborted)\n"
            schedule.loc[ts, thread] = "thread finish txn %s %s"%(txn, thread)
            
    elif "[row_" in tokens[0]:
            #return
            if "add txn" in line:
                    #[row_ww] add txn 1 type 1 to owners of row 1449388428864366
                    txn = tokens[3]
                    lock = get_lock(tokens[5])
                    row = tokens[-1]
                    thread = get_thd(txn)
                    if row not in locktable:
                        locktable[row] = {}
                        locktable[row]['owners'] = []
                        locktable[row]['waiters'] = []
                        if clv:
                            locktable[row]['retired'] = []
                    locktable[row][tokens[7]].append((txn, lock))
                    if "owner" in tokens[7]:
                        schedule.loc[ts, thread] = "acquire %s lock %s"%(lock, row)
                    elif "waiter" in tokens[7]:
                        schedule.loc[ts, thread] = "wait %s lock %s"%(lock, row)
                    elif "retired" in tokens[7]:
                        schedule.loc[ts, thread] = "retire %s lock %s"%(lock, row)
            elif "rm another txn" in line:
                    # [row_clv] txn %lu rm another txn %lu from owners of row %lu
                    wounder = tokens[2]
                    txn = tokens[6]
                    row = tokens[-1]
                    place = tokens[8]
                    thread, _ = rm_txn(row, place, txn)
                    if "owner" in place:
                        schedule.loc[ts, thread] = "removed lock %s from owners by %s"%(row, wounder)
                    elif "retired" in place:
                        schedule.loc[ts, thread] = "removed lock %s from retired by %s"%(row, wounder)
            elif "rm txn" in line:
                    txn = tokens[3]
                    row = tokens[-1]
                    thread, _ = rm_txn(row, tokens[5], txn)
                    if "owner" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from owners"%row
                    elif "waiter" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from waiters"%row
                    elif "retired" in tokens[5]:
                        schedule.loc[ts, thread] = "release lock %s from retired"%row
            elif "from waiter to owner" in line or "from waiters to owners" in line:
                    # [row_ww] bring 0 from waiter to owner of row 1454103086539417
                    txn = tokens[2]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    thread, entry = rm_txn(row, "waiters", txn)
                    locktable[row]["owners"].append(entry)
                    schedule.loc[ts, thread] = "acquire lock %s"%row
            elif "from owners to retired" in line:
                    # [row_clv] move txn 0 from owners to retired type 0 of row 1459774078579136
                    txn = tokens[3]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    lock = get_lock(tokens[-4])
                    locktable[row]["retired"].append((txn, lock))
                    schedule.loc[ts, thread] = "retire lock %s"%row
            elif "abort txn" in line:
                # [row_ww] txn 8 abort txn 6 on row 10
                wound = tokens[2]
                abort = tokens[-4]
                row = tokens[-1]
                txn = wound
                schedule.loc[ts, get_thd(wound)] = "wound %s on %s"%(abort, row)
                schedule.loc[ts, get_thd(abort)] = "%s wounded by %s"%(abort,wound)
    elif "[txn]" in tokens[0]:
        if "set txn" in line and "to abort" in line:
            wounder = tokens[1]
            txn = tokens[4]
            schedule.loc[ts, get_thd(txn)] = "txn %s set abort by %s"% (txn, wounder)
            schedule.loc[ts, get_thd(wounder)] = "wound txn %s"%txn
        elif "is set to" in line:
            # [txn] txn %lu is set to commited
            txn = tokens[2]
            status = tokens[-1]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "txn %s set %s"%(txn, status) 
            if status != "commited":
                barriers[txn] = -1
        elif "barrier" in line:
            # "[txn] increment barrier for txn %lu\n"
            txn = tokens[-1]
            thread = get_thd(txn)
            if tokens[1] == "increment":
                barriers[txn] += 1
            elif tokens[1] == "decrement":
                barriers[txn] -= 1
            schedule.loc[ts, thread] = "txn %s %s barrier, now should be %d"%(txn, tokens[1], barriers[txn])
        elif "# bariers" in line:
            # [txn] # bariers of txn 1 = 0
            txn = tokens[-3]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "pre-commit: txn %s's barriers = %d"%(txn, int(tokens[-1]))
            assert(int(tokens[-1]) == barriers[txn])
#     if txn in ["9"] or "txn 9" in line:
#         print("[{}]: {}".format(ts, line))

In [85]:
output = open('debug.out')

In [86]:
locktable = {} # row_id: { owners: {}, waiters: {}, (opt) retired: {}}
barriers = {}
schedule = pd.DataFrame()
ts = 0

In [87]:
for line in tqdm(output):
    try:
        parse(line.strip())
        ts += 1
    except:
        print(line)
        raise

10620it [00:10, 1029.78it/s]


In [88]:
import numpy as np
for i in schedule:
    print("thread {}: {}".format(i, schedule[i][schedule[i].notnull()].values[-1]))

thread 0: removed lock 536900749115690 from retired by 217
thread 2: remove lock 536900824811266 from owners
thread 1: retire lock 536901130457102


In [91]:
find_locktable_with_txn('59')

536900749115690
{'owners': [], 'waiters': [], 'retired': [('59', 'EX'), ('256', 'EX')]}
536900749123440
{'owners': [], 'waiters': [], 'retired': [('59', 'SH')]}
536901320769112
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536900743436293
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536901135006095
{'owners': [], 'waiters': [], 'retired': [('59', 'SH')]}
536900743459196
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536901135100100
{'owners': [], 'waiters': [], 'retired': [('59', 'SH')]}
536900724123117
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536901067982348
{'owners': [], 'waiters': [], 'retired': [('59', 'SH')]}
536900680504323
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536900911435077
{'owners': [], 'waiters': [], 'retired': [('59', 'SH')]}
536900656160397
{'owners': [], 'waiters': [], 'retired': [('256', 'EX'), ('59', 'EX')]}
536900652796333
{'owners': [], 'waiters': [], 'retired': [('59', 'EX')]}
536900811944619
{'own

In [ ]:
# get_schedule(299, 349, [1,3])

In [35]:
schedule.to_csv('sample_schedule.csv')

In [89]:
schedule[0][schedule[0].notnull()]

12                                           start txn 0
22                          wait SH lock 536900749115690
24                          acquire lock 536900749115690
26               remove lock 536900749115690 from owners
27                           retire lock 536900749115690
                              ...                       
9250                         retire lock 536900749124528
9296          txn 138 decrement barrier, now should be 0
9318                            txn 138 set abort by 217
9321                                  138 wounded by 217
9322    removed lock 536900749115690 from retired by 217
Name: 0, Length: 2548, dtype: object

In [90]:
for b in barriers:
    if barriers[b] > 0:
        print("{} = {} at thread {}".format(b, barriers[b], int(b)%NUM_THREAD))

59 = 1 at thread 2
256 = 1 at thread 1
